In [3]:
import pandas as pd

movies = pd.read_csv("movies.csv")

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [6]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [7]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]

    return results

In [10]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [11]:
movie_id = 1288

movie = movies[movies["movieId"] == movie_id]

In [12]:
ratings = pd.read_csv("ratings.csv")

In [13]:
ratings.dtypes

,0
userId,int64
movieId,int64
rating,float64
timestamp,int64


In [14]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [15]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [16]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [17]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [18]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [19]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [20]:
rec_percentages

,similar,all
movieId,,
1288,1.000000,0.045595
296,0.511765,0.282687
1136,0.488235,0.125386
858,0.488235,0.202494
608,0.447059,0.137455
...,...,...
58559,0.102941,0.136114
1084,0.102941,0.016629
1293,0.102941,0.027223


In [21]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [22]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [23]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
1255,1.000000,0.045595,21.932353,1288,This Is Spinal Tap (1984),Comedy,This Is Spinal Tap 1984
1409,0.114706,0.010862,10.560022,1449,Waiting for Guffman (1996),Comedy,Waiting for Guffman 1996
2198,0.135294,0.014885,9.089083,2289,"Player, The (1992)",Comedy|Crime|Drama,Player The 1992
3808,0.167647,0.019579,8.562631,3911,Best in Show (2000),Comedy,Best in Show 2000
160,0.102941,0.013410,7.676324,162,Crumb (1994),Documentary,Crumb 1994
1611,0.176471,0.023736,7.434696,1673,Boogie Nights (1997),Drama,Boogie Nights 1997
1358,0.250000,0.033660,7.427291,1394,Raising Arizona (1987),Comedy,Raising Arizona 1987
1197,0.200000,0.027223,7.346798,1228,Raging Bull (1980),Drama,Raging Bull 1980
1863,0.108824,0.015288,7.118395,1952,Midnight Cowboy (1969),Drama,Midnight Cowboy 1969
1215,0.102941,0.014751,6.978476,1248,Touch of Evil (1958),Crime|Film-Noir|Thriller,Touch of Evil 1958


In [24]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [25]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()